In [1]:
import os
import glob
import json
from tqdm.auto import tqdm
import numpy as np
from collections import deque
import cv2
from collections import deque


In [16]:
POSTPROCESSING = True
# change keyframe_dirs to the directory where of whether the original/postprocessed keyframes are stored
keyframes_dirs = [
        '/kaggle/input/postprocessed-data/postprocessed_keyframes',
        ]

video_dirs = [
    '/kaggle/input/aic2024-videos-part2',
#     '/kaggle/input/aic2024-videos-part1-1',
#     '/kaggle/input/aic2024-videos-part1',

]

keyframes_dirs_og = [
    '/kaggle/input/aic2024-compressed-keyframes-p2',
#     '/kaggle/input/aic2024-compressed-keyframes-p1-1',
#     '/kaggle/input/aic2024-compressed-keyframes-p1-2'
]

metadata_dir = '/kaggle/input/media-info/media-info'
des_path = '/kaggle/working/'
scene_dirs = [
    '/kaggle/input/get-scenes/scenes',
#     '/kaggle/input/scenes/scenes'
]

In [17]:
# deleted_info = json.load(open('/kaggle/input/deleted-info-p1/deleted_info.json'))
deleted_info = json.load(open('/kaggle/input/deleted-info-p1/deleted_info2.json'))

all_keyframe_paths_og = dict()
for keyframes_dir in keyframes_dirs_og:
    for batch_key in sorted(os.listdir(keyframes_dir)):
        batch = batch_key.split('_')[0]
        batch_dir = os.path.join(keyframes_dir, batch_key, batch_key)

        all_keyframe_paths_og[batch] = dict()
        for video_key in sorted(os.listdir(batch_dir)):
            video_dir = os.path.join(batch_dir, video_key)

            keyframes = sorted(glob.glob(f'{video_dir}/*.webp'))
            all_keyframe_paths_og[batch][video_key] = keyframes
            
            print(f"added {batch}_{video_key} with {len(keyframes)} keyframes")

added L13_V001 with 909 keyframes
added L13_V002 with 927 keyframes
added L13_V003 with 915 keyframes
added L13_V004 with 1143 keyframes
added L13_V005 with 945 keyframes
added L13_V006 with 1149 keyframes
added L13_V007 with 855 keyframes
added L13_V008 with 768 keyframes
added L13_V009 with 774 keyframes
added L13_V010 with 1056 keyframes
added L13_V011 with 972 keyframes
added L13_V012 with 761 keyframes
added L13_V013 with 1137 keyframes
added L13_V014 with 945 keyframes
added L13_V015 with 747 keyframes
added L13_V016 with 969 keyframes
added L13_V017 with 750 keyframes
added L13_V018 with 927 keyframes
added L13_V019 with 966 keyframes
added L13_V020 with 939 keyframes
added L13_V021 with 831 keyframes
added L13_V022 with 795 keyframes
added L13_V023 with 816 keyframes
added L13_V024 with 711 keyframes
added L13_V025 with 897 keyframes
added L13_V026 with 1101 keyframes
added L13_V027 with 876 keyframes
added L13_V028 with 918 keyframes
added L13_V029 with 813 keyframes
added L13

In [19]:
all_videos = dict()
for vdir in video_dirs:
    for batch_key in tqdm(os.listdir(vdir)):
        
        if 'part1' in vdir:
            batch_path = os.path.join(vdir, batch_key, 'video')
        else:
            batch_path = os.path.join(vdir, batch_key, batch_key, 'video')
        
        
        for video_path in tqdm(sorted(glob.glob(os.path.join(batch_path, '*.mp4')))):
            video_key = video_path.split('/')[-1].replace('.mp4', '')
            cap = cv2.VideoCapture(video_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            
            metadata = json.load(open(os.path.join(metadata_dir, f'{video_key}.json'), encoding='utf-8'))
            
            all_videos[video_key] = metadata
            all_videos[video_key]['fps'] = fps

scenes_id2info = dict()
for scene_dir in scene_dirs:
    for batch in sorted(os.listdir(scene_dir)):
        batch_path = os.path.join(scene_dir, batch)
        scenes_id2info[batch] = {}

        for video in sorted(os.listdir(batch_path)):
            video_id = video.split('.')[0]
            video_path = os.path.join(batch_path, video)
            video_fps = all_videos[batch + '_' + video_id]['fps']

            video_scene_info = json.load(open(video_path, 'r'))
            scenes_id2info[batch][video_id] = {}

            for i, scene_bound in enumerate(video_scene_info):
                scenes_id2info[batch][video_id][str(i)] = {
                    'shot_range': scene_bound,
                    'shot_time': [scene_bound[0]/video_fps, scene_bound[1]/video_fps],
                    'lst_keyframe_ids': []
                }
            
all_scene_paths = dict()
for scene_dir in scene_dirs:
    for batch in sorted(os.listdir(scene_dir)):
        batch = batch.split('_')[-1]
        batch_dir = os.path.join(scene_dir, batch)

        all_scene_paths[batch] = dict()
        for scene in sorted(glob.glob(f'{batch_dir}/*.json')):
            video_name = os.path.basename(scene).split('.')[0]
            all_scene_paths[batch][video_name] = scene

In [20]:
def get_frame_numbers(scenes):
    frame_numbers = np.zeros(len(scenes) * 3, dtype=int)
    frame_numbers[0::3] = scenes[:, 0]  # start frames
    frame_numbers[1::3] = (scenes[:, 0] + scenes[:, 1]) // 2
    frame_numbers[2::3] = scenes[:, 1]  # end frames
    frame_numbers = np.unique(frame_numbers)
    
    return frame_numbers

In [23]:
global_index = 229329 + 1
id2img_fps = []

all_keyframe_paths = dict()
all_video_info = dict()

for keyframes_dir in keyframes_dirs:
    for batch_key in sorted(os.listdir(keyframes_dir)):
        batch_path = os.path.join(keyframes_dir, batch_key) 
        
        all_keyframe_paths[batch_key] = dict()
        all_video_info[batch_key] = dict()
        
        print('Processing batch', batch_key.replace('_extra', ''))
        for video_id in tqdm(sorted(os.listdir(batch_path))):
            video_dir = os.path.join(batch_path, video_id)

            scene_batch_key = batch_key.replace('_extra', '')
            
            print(scene_batch_key, video_id)
            scenes = json.load(open(all_scene_paths[scene_batch_key][video_id], 'r'))
            scenes = np.array(scenes)
            
            frame_numbers = get_frame_numbers(scenes)
            
            keyframes = all_keyframe_paths_og[batch_key][video_id]
            deleted_frames_idx = deleted_info[batch_key][video_id]['frm_number']
            all_keyframe_numbers = list(range(len(keyframes)))
            deleted_keyframe_numbers = deleted_info[batch_key][video_id]['frm_number']
            desired_keyframe_numbers = list(set(all_keyframe_numbers) - set(deleted_keyframe_numbers))
            
            scene_track = 0
            scenes = deque(scenes)
            
            all_images = sorted(glob.glob(f'{video_dir}/*.webp'))
            frame_numbers = frame_numbers[desired_keyframe_numbers]
            
            assert len(all_images) == len(frame_numbers), f"Length mismatch: all_images: {len(all_images)}, frame_numbers: {len(frame_numbers)}."
            
            all_keyframe_paths[batch_key][video_id] = {}
        
            current_video = all_videos[f'{batch_key}_{video_id}']
            all_video_info[batch_key][video_id] = {
                'publish_date': current_video['publish_date'],
                "fps": current_video["fps"],
                "watch_url": current_video["watch_url"],
                "frm_numbers": frame_numbers.tolist(),
                "files": desired_keyframe_numbers
            }
            
            for filename, frame_num in zip(desired_keyframe_numbers, frame_numbers):
                all_keyframe_paths[batch_key][video_id][int(filename)] = {
#                     'frm_number': str(frame_num),
                    'frm_id': str(global_index)
                }
                            
            for image_path, frame_number in zip(all_images, frame_numbers):
                keyframes_root = os.path.basename(keyframes_dir)
                data_index = image_path.find(keyframes_root)
                if data_index != -1:
                    image_path = image_path[data_index:]
                    image_path = image_path.replace(keyframes_root, '', 1).replace('_extra', '')       

                info = {
                    "img_path": image_path,
                    "scene_id": f'/{scene_batch_key}/{video_id}/{str(scene_track)}',
                    'frm_number': str(frame_number)
                }
                scenes_id2info[scene_batch_key][video_id][str(scene_track)]['lst_keyframe_ids'].append(global_index)
                id2img_fps.append(info)
                
                global_index += 1

                if frame_number >= scenes[0][1]:
                    scene_track += 1
                    scenes.popleft()
            
                    
id2img_fps = dict(enumerate(id2img_fps))

Processing batch L13


  0%|          | 0/1 [00:00<?, ?it/s]

L13 L13_extra


KeyError: 'L13_extra'

In [ ]:
with open(f'{des_path}/id2img_fps.json', 'w') as f:
    f.write(json.dumps(id2img_fps, indent=2))

with open(f'{des_path}/scene_id2info.json', 'w') as f:
    f.write(json.dumps(scenes_id2info, indent=2))

def custom_serializer(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    if isinstance(obj, np.float32):
        return float(obj)
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")


with open(f'{des_path}/all_keyframe_map.json', 'w') as f:
    json.dump(all_keyframe_paths, f, indent=2)

with open(f'{des_path}/all_video_info.json', 'w') as f:
    json.dump(all_video_info, f)
    
global_index

## **Encode Context**

In [13]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.2 MB/s eta 0:00:0000:0100:01m


In [14]:
import os
import sys
import json
import torch
import faiss
import tqdm.auto as tqdm
import numpy as np
from typing import List, Dict, Any, Tuple
import gc
import glob
import torch.nn.functional as F
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import scipy as sp

In [15]:
def load_context(clean_data_paths):
    context = []
    data_paths = []
    
    cxx_data_paths = glob.glob(clean_data_paths)
    cxx_data_paths.sort()
    for cxx_data_path in cxx_data_paths:
        data_path = glob.glob(cxx_data_path + '/*.txt')
        data_path.sort(reverse=False, key=lambda s:int(s[-7:-4]))
        data_paths += data_path
    for path in data_paths:
        with open(path, 'r', encoding='utf-8') as f:
            data = f.readlines()
            data = [item.strip() for item in data]
            context += data
    return context

def preprocess_text(text:str):
    text = text.lower()
    # keep letter and number remove all remain
    reg_pattern = '[^a-z0-9A-Z_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ\\s]'
    output = re.sub(reg_pattern, '', text)
    output = output.strip()
    return output

In [18]:
# sparse_context_types = ['bbox', 'class', 'number', 'tag']
dense_context_types = []
sparse_context_types = ['tag', 'ocr']

all_context_types = sparse_context_types + dense_context_types

out_dir = '/kaggle/working'

in_paths = {
    'bbox':   'bboxes_encoded/*',
    'class':  'classes_encoded/*',
    'color':  'colors_encoded/*',
    'tag':    '/kaggle/input/postprocessed-data/tag-encoded-p1/tags_encoded/*',
    'number': 'number_encoded/*',
    'ocr': 
}

os.makedirs(os.path.join(out_dir, 'transform'), exist_ok=True)
os.makedirs(os.path.join(out_dir, 'context'), exist_ok=True)

out_paths = {
    type: f"context/{type}_encoded.npz" for type in sparse_context_types
}
out_paths.update({
    type: f"context/{type}_encoded.bin" for type in dense_context_types
})

transform_paths = {
    type: f"transform/{type}_tfidf_transform.pkl" for type in all_context_types
}

ngram_range = (1, 1)
save_corpus_path = '/kaggle/working/tag_corpus.txt'

In [19]:
for type in all_context_types:
    clean_data_paths = in_paths[type]
    context = load_context(clean_data_paths)
    
    tfidf_transform = TfidfVectorizer(input = 'content', ngram_range = ngram_range, token_pattern=r"(?u)\b[\w\d]+\b")
    context_matrix = tfidf_transform.fit_transform(context).tocsr()

    transform_path = os.path.join(out_dir, transform_paths[type])
    context_path = os.path.join(out_dir, out_paths[type])
    
    if type == 'tag':
        tag_corpus = tfidf_transform.get_feature_names_out()
        corpus = [' '.join(words.split('_')) for words in tag_corpus]
        corpus = '\n'.join(corpus) + '\n'
        with open(save_corpus_path, 'w') as f:
            f.write(corpus)
            
    total_elements = context_matrix.shape[0] * \
        context_matrix.shape[1]
    non_zero_elements = context_matrix.nnz
    sparsity = 1 - (non_zero_elements / total_elements)
    # A value close to 1 indicates high sparsity
    print(f"Sparsity of : {sparsity:.4f}")
    
    with open(transform_path, 'wb') as f:
        pickle.dump(tfidf_transform, f)
        
    if type in sparse_context_types:
        sp.sparse.save_npz(context_path, context_matrix)
    else:
        context_matrix = context_matrix.toarray()
        context_matrix = np.float32(context_matrix)
        feature_shape = context_matrix.shape[1]
        
        index = faiss.IndexFlatIP(feature_shape)
        faiss.write_index(index, context_path)

Sparsity of : 0.9973


## **Encode CLIP**

In [23]:
from tqdm.auto import tqdm

In [24]:
clip = np.load(f"/kaggle/input/postprocessed-data/clip-features-2-p1/CLIP_model2_features/L01/V001.npy")
clip.shape

feature_shape = clip.shape[1]
features_dir = "/kaggle/input/postprocessed-data/clip-features-2-p1/CLIP_model2_features"

index = faiss.IndexFlatIP(feature_shape)

for data_part in tqdm(sorted(os.listdir(features_dir))):
    for feature_path in tqdm(sorted(glob.glob(os.path.join(features_dir, data_part) +'/*.npy'))):
        feats = np.load(feature_path)
        for feat in feats:
            feat = feat.astype(np.float32).reshape(1,-1)
            index.add(feat)

faiss.write_index(index, f"/kaggle/working/model2.bin")

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]